In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [6]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()



In [9]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [9]:
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
#x = layers.Flatten()(last_output)
x = layers.Flatten()

# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)           

model = x 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
pass

In [13]:
import zipfile

local_zip = '/content/drive/My Drive/675484_1187790_bundle_archive.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

base_dir = '/tmp/Face Mask Dataset'

train_dir = os.path.join( base_dir, 'Train')
validation_dir = os.path.join( base_dir, 'Validation')

train_cats_dir = os.path.join(train_dir, 'WithMask')
train_dogs_dir = os.path.join(train_dir, 'WithoutMask')
validation_cats_dir = os.path.join(validation_dir, 'WithMask')
validation_dogs_dir = os.path.join(validation_dir, 'WithoutMask')

In [15]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
    )


train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(150,150),
)     

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [23]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 500,
    validation_data = validation_generator, 
    validation_steps = 40,
    epochs = 30)

Epoch 1/30
500/500 [==============================] - 64s 128ms/step - loss: 0.1946 - accuracy: 0.9248 - val_loss: 0.1087 - val_accuracy: 0.9688
Epoch 2/30
500/500 [==============================] - 63s 126ms/step - loss: 0.1801 - accuracy: 0.9318 - val_loss: 0.0902 - val_accuracy: 0.9750
Epoch 3/30
500/500 [==============================] - 63s 126ms/step - loss: 0.1604 - accuracy: 0.9408 - val_loss: 0.0901 - val_accuracy: 0.9787
Epoch 4/30
500/500 [==============================] - 63s 126ms/step - loss: 0.1569 - accuracy: 0.9428 - val_loss: 0.0794 - val_accuracy: 0.9775
Epoch 5/30
500/500 [==============================] - 63s 126ms/step - loss: 0.1462 - accuracy: 0.9467 - val_loss: 0.0727 - val_accuracy: 0.9762
Epoch 6/30
500/500 [==============================] - 63s 127ms/step - loss: 0.1440 - accuracy: 0.9477 - val_loss: 0.0747 - val_accuracy: 0.9787
Epoch 7/30
500/500 [==============================] - 63s 127ms/step - loss: 0.1416 - accuracy: 0.9474 - val_loss: 0.0662 - val_ac

In [24]:
import numpy as np
from keras.preprocessing import image

img_width, img_height = 150, 150
img = image.load_img('/tmp/Face Mask Dataset/Test/WithMask/1163.png', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

In [25]:
model.predict(img)

array([[0.]], dtype=float32)

In [26]:
img_width, img_height = 150, 150
img1 = image.load_img('/tmp/Face Mask Dataset/Test/WithoutMask/1177.png', target_size = (img_width, img_height))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1, axis = 0)

In [27]:
model.predict(img1)

array([[1.]], dtype=float32)

In [28]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [29]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                       drive = GoogleDrive(gauth)


SyntaxError: ignored

In [31]:
model.save('/tmp/model.h5')